<img src="img/se4g_pic1.png">

In [1]:
from se4g_helper import *


In [2]:
# Create an instance of the FoliumMap class
folium_map =

In [3]:
# Display the filter buttons
selected_pollutant, selected_datetime =  FoliumMap().select_filters()

Dropdown(description='Select pollutant:', options=('SO2', 'NO', 'NO2', 'CO', 'PM10'), value='SO2')

Dropdown(description='Select datetime:', options=('2023-06-09 21:00:00+01:00', '2023-06-17 09:00:00+01:00', '2…

In [4]:
# Attach the callback function to the dropdowns' value change event
selected_pollutant.observe(folium_map.update_maps, 'value')
selected_datetime.observe(folium_map.update_maps, 'value')

# Initial map display
folium_map.update_maps(None) 

In [5]:
# Create an instance of the Dashboard class and run the application
dashboard = Dashboard().run()

Dash is running on http://127.0.0.1:8050/



When running in JupyterLab, with the `jupyterlab-dash` extension, setting `mode="jupyterlab"` will open the app in a tab in JupyterLab.

In [7]:
app.run_server(mode="jupyterlab")

Dash is running on http://127.0.0.1:8050/



<img src="img/se4g_pic2.png">